In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time

import re  
import nltk

from nltk.corpus import stopwords
from nltk.corpus import RegexpTokenizer as regextoken
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import pickle

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn.decomposition import PCA
from __future__ import print_function

In [2]:
os.getcwd()

'/Users/z066910/Desktop/PHDHackathon'

#### Import Test data

In [3]:
testds=pd.read_csv('GrievancesData_Test.csv')

In [4]:
testds.dtypes

GrievanceID             object
BankID                  object
State                   object
DateOfGrievance         object
Grievance_Category      object
GrievanceDescription    object
LineOfBusiness          object
ResolutionComments      object
Disputed                object
DateOfResolution        object
dtype: object

In [5]:
testds.describe()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
count,27954,27954,27879,27954,27954,27954,27954,27954,27954,27954
unique,27954,22,61,1168,59,24137,4,4,2,1198
top,GID436025,Bank5373,State9,2014-01-19,"Loan servicing, payments, escrow account",I was shocked when I reviewed my credit repor...,Credit card,Closed with explanation,No,2014-02-19
freq,1,8221,3839,727,4403,6,9561,23505,22524,291


In [6]:
testds.head(1)

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
0,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30


In [7]:
testds.tail(1)

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
27953,GID502376,Bank5372,State22,2015-08-23,Problem with a credit reporting company's inve...,See attached document showing payments still ...,Mortgage,Closed with explanation,No,2015-09-23


In [8]:
print(testds.columns)

Index([u'GrievanceID', u'BankID', u'State', u'DateOfGrievance',
       u'Grievance_Category', u'GrievanceDescription', u'LineOfBusiness',
       u'ResolutionComments', u'Disputed', u'DateOfResolution'],
      dtype='object')


In [9]:
#Check for the shape (no.of rows,columns) of the data
print(testds.shape)

(27954, 10)


In [10]:
#Check for missing values in the data
print(testds.isnull().sum())

GrievanceID              0
BankID                   0
State                   75
DateOfGrievance          0
Grievance_Category       0
GrievanceDescription     0
LineOfBusiness           0
ResolutionComments       0
Disputed                 0
DateOfResolution         0
dtype: int64


In [24]:
#Frequency Count
#testds.groupby('BankID').size()
#testds.groupby('GrievanceID').size()
#testds.groupby('State').size()
#testds.groupby('DateOfGrievance').size()
#testds.groupby('GrievanceDescription').size()
#No.o unique text 24137
#testds.groupby('LineOfBusiness').size()
#testds.groupby('ResolutionComments').size()
#Closed with explanation -> # texts 23505
#testds.groupby('Disputed').size()
# Disputed = Yes -> # grievances 5430
#testds.groupby('DateOfResolution').size()

In [25]:
#Remove duplicate records

#Deleting if records are repeting just with different grievance id
testdf=testds.drop_duplicates(['BankID','State','DateOfGrievance','Grievance_Category','GrievanceDescription','LineOfBusiness','ResolutionComments','Disputed','DateOfResolution'],keep='first')

In [27]:
testdf.shape

(24165, 10)

In [29]:
#Reset the index of the deduped data
testdf.reset_index()
testdf1=testdf.reset_index(drop=True)
testdf1.head(1)

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
0,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30


In [30]:
#Data for text mining
datatxt= testdf1[['GrievanceID','GrievanceDescription']]

In [31]:
#Define function to preprocess text
def text_to_words( text ):
    start = time.time()
    #letters_only = re.sub("[^a-zA-Z0-9$%!]", " ", text) 
    letters_only = re.sub("[^a-zA-Z]", " ", text) 
    words = letters_only.lower().split()
    stop_words = stopwords.words('english')
    #Extending the stop words list by adding data specific unwanted words to the list
    stop_words.extend(['xx','xxx','xxxx','bank america','bank','citi','ocwen','ocwens','gl','rp','fitbit','cement','mls','lfs','reo','hhf','be','di','pt','ax','sp','america','american','bankamericard','americans','americas','aaa','aa','aafes','us','usa','usbank'])
    stops = set(stop_words)  
    lm=WordNetLemmatizer()
    meaningful_words = [lm.lemmatize(w,pos='v') for w in words if not w in stops]
    
    #Convert plurals to singular
    #postg=pos_tag(meaningful_words)
    #newwords=[w[0].rstrip("s") if w[1]=='NNS' else w[0] for w in postg]
    
    #ps = PorterStemmer()
    #meaningful_words = [ps.stem(w) for w in words if not w in stops]
    
    return( " ".join( meaningful_words )) 
    #return( " ".join( newwords )) 
    
    end = time.time()
    print("Time elapsed for execution = ", end - start)

In [32]:
#No.of texts to process
num_texts = datatxt["GrievanceDescription"].size
print(num_texts)

24165


In [34]:

#Define function to clean text
def cleantxt(IP,rec):
    #Initialize an empty list to hold the clean texts
    clean_reviews = []
    start = time.time()
    #Loop over each text
    for i in range(0, rec):
        #Call function for each text, and add the result to the list of clean reviews
        clean_reviews.append(text_to_words(IP["GrievanceDescription"][i]))
    
    end = time.time()
    print("Time elapsed for execution = ", end - start)
    return clean_reviews                          

In [37]:
#Call clean text function to create corpus of cleaned text, all data (train)
Tclean_reviews=cleantxt(datatxt,rec=num_texts)
Tclean_reviews

Time elapsed for execution =  29.7082738876


[u'currently mortgage flag star payment late reach flag star loan year first time payment late due personal issue currently current payments receive notice loan go service interest loan transfer company address issue look company nothing horror stories post avoid loan transfer',
 u'auto finance repossess subsequently sell car husband chapter bankruptcy inquire regain possession rectify account tell call back bankruptcy call back already sell car credit report show repossession recovery amount owe give chance regain possession car take payments',
 u'fell behind payments back follow really unreasonable modification modification place loan years years old never live pay months modification payment go month fell behind due fact health problems medical condition go time also lose month income due family members move home contribute household time receive notice mortgage company payment date letter contact point contact person inform difficulty need time contact mortgage company man answer p

## Feature Extraction for test data

#### Join the clean descriptions with the original data for creating features out of it

In [41]:
#Convert the clean_reviews list to dataframe
Tclean_reviews
Tclean_reviews_df = pd.DataFrame({'GrievanceDesc_clean':Tclean_reviews})
Tclean_reviews_df.shape
Tclean_reviews_df.head(2)

#Join the cleaned reviews with orignal data

frames = [Tclean_reviews_df, testdf1]
df2 = pd.concat(frames,axis=1)

df2=  df2[['GrievanceDesc_clean','GrievanceID','LineOfBusiness','ResolutionComments','Disputed']]
#df2.shape
#df2.head(1)
#df1
num_texts = df2['GrievanceDesc_clean'].size
num_texts

24165

#### Import the BOW for creating features

In [39]:
BOWcorpus=pd.read_csv('BOW_NEW.csv')

In [40]:
#Import Pandas SQL library
from pandasql import *
import pandas as pd

#### Feature -> BusinessPractices

In [42]:
BOW = BOWcorpus[BOWcorpus["BusinessPractices"].notnull()]
BOW=BOW[["BusinessPractices"]]

num_keywords = BOW["BusinessPractices"].size
num_keywords

BOW = BOW.rename(columns={'BusinessPractices': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'BusinessPractices'})

print("Look at the distribution")
print(df3['BusinessPractices'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [df2, df3['BusinessPractices']]
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    17899
1     6266
Name: BusinessPractices, dtype: int64


In [43]:
## Feature Communication

BOW = BOWcorpus[BOWcorpus["Communication"].notnull()]
BOW=BOW[["Communication"]]

num_keywords = BOW["Communication"].size
num_keywords

BOW = BOW.rename(columns={'Communication': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'Communication'})

print("Look at the distribution")
print(df3['Communication'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['Communication']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
1    14374
0     9791
Name: Communication, dtype: int64


In [44]:
##CIBIL Score

BOW = BOWcorpus[BOWcorpus["CIBILScore"].notnull()]
BOW=BOW[["CIBILScore"]]

num_keywords = BOW["CIBILScore"].size
num_keywords

BOW = BOW.rename(columns={'CIBILScore': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'CIBILScore'})

print("Look at the distribution")
print(df3['CIBILScore'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['CIBILScore']]
df2 = pd.concat(frames,axis=1)

FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    19755
1     4410
Name: CIBILScore, dtype: int64


In [45]:
##Late fee and interest charges

BOW = BOWcorpus[BOWcorpus["LateFeeInterestCharges"].notnull()]
BOW=BOW[["LateFeeInterestCharges"]]

num_keywords = BOW["LateFeeInterestCharges"].size
num_keywords

BOW = BOW.rename(columns={'LateFeeInterestCharges': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'LateFeeInterestCharges'})

print("Look at the distribution")
print(df3['LateFeeInterestCharges'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['LateFeeInterestCharges']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    16776
1     7389
Name: LateFeeInterestCharges, dtype: int64


In [46]:
##Payments

BOW = BOWcorpus[BOWcorpus["Payments"].notnull()]
BOW=BOW[["Payments"]]

num_keywords = BOW["Payments"].size
num_keywords

BOW = BOW.rename(columns={'Payments': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'Payments'})

print("Look at the distribution")
print(df3['Payments'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['Payments']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    16045
1     8120
Name: Payments, dtype: int64


In [47]:
##Close and Cancel
BOW = BOWcorpus[BOWcorpus["CloseCancel"].notnull()]
BOW=BOW[["CloseCancel"]]

num_keywords = BOW["CloseCancel"].size
num_keywords

BOW = BOW.rename(columns={'CloseCancel': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'CloseCancel'})

print("Look at the distribution")
print(df3['CloseCancel'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['CloseCancel']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    18151
1     6014
Name: CloseCancel, dtype: int64


In [48]:
##Purchase
BOW = BOWcorpus[BOWcorpus["purchase"].notnull()]
BOW=BOW[["purchase"]]

num_keywords = BOW["purchase"].size
num_keywords

BOW = BOW.rename(columns={'purchase': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'purchase'})

print("Look at the distribution")
print(df3['purchase'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['purchase']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    21674
1     2491
Name: purchase, dtype: int64


In [49]:
##Delay or reccur

BOW = BOWcorpus[BOWcorpus["DelayRecur"].notnull()]
BOW=BOW[["DelayRecur"]]

num_keywords = BOW["DelayRecur"].size
num_keywords

BOW = BOW.rename(columns={'DelayRecur': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'DelayRecur'})

print("Look at the distribution")
print(df3['DelayRecur'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['DelayRecur']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
1    16999
0     7166
Name: DelayRecur, dtype: int64


In [50]:
##Legal
BOW = BOWcorpus[BOWcorpus["legal"].notnull()]
BOW=BOW[["legal"]]

num_keywords = BOW["legal"].size
num_keywords

BOW = BOW.rename(columns={'legal': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'legal'})

print("Look at the distribution")
print(df3['legal'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['legal']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    15256
1     8909
Name: legal, dtype: int64


In [51]:
##Mktng and Offer

BOW = BOWcorpus[BOWcorpus["Mkting_Offer_Reward"].notnull()]
BOW=BOW[["Mkting_Offer_Reward"]]

num_keywords = BOW["Mkting_Offer_Reward"].size
num_keywords

BOW = BOW.rename(columns={'Mkting_Offer_Reward': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'Mkting_Offer_Reward'})

print("Look at the distribution")
print(df3['Mkting_Offer_Reward'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['Mkting_Offer_Reward']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    19337
1     4828
Name: Mkting_Offer_Reward, dtype: int64


In [52]:
##Payments

BOW = BOWcorpus[BOWcorpus["PaymentIssue"].notnull()]
BOW=BOW[["PaymentIssue"]]

num_keywords = BOW["PaymentIssue"].size
num_keywords

BOW = BOW.rename(columns={'PaymentIssue': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'PaymentIssue'})

print("Look at the distribution")
print(df3['PaymentIssue'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['PaymentIssue']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
1    15813
0     8352
Name: PaymentIssue, dtype: int64


In [53]:
##Mortgage

BOW = BOWcorpus[BOWcorpus["MortgageBroadly"].notnull()]
BOW=BOW[["MortgageBroadly"]]

num_keywords = BOW["MortgageBroadly"].size
num_keywords

BOW = BOW.rename(columns={'MortgageBroadly': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'MortgageBroadly'})

print("Look at the distribution")
print(df3['MortgageBroadly'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['MortgageBroadly']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    16085
1     8080
Name: MortgageBroadly, dtype: int64


In [54]:
##Theft n Fraud

BOW = BOWcorpus[BOWcorpus["TeftFraudThreat"].notnull()]
BOW=BOW[["TeftFraudThreat"]]

num_keywords = BOW["TeftFraudThreat"].size
num_keywords

BOW = BOW.rename(columns={'TeftFraudThreat': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'TeftFraudThreat'})

print("Look at the distribution")
print(df3['TeftFraudThreat'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['TeftFraudThreat']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
0    20705
1     3460
Name: TeftFraudThreat, dtype: int64


In [55]:
## UFDP

BOW = BOWcorpus[BOWcorpus["UFDP"].notnull()]
BOW=BOW[["UFDP"]]

num_keywords = BOW["UFDP"].size
num_keywords

BOW = BOW.rename(columns={'UFDP': 'list'})

pysqldf = lambda q: sqldf(q, globals())

q = """SELECT df2.*,BOW.* FROM df2 LEFT JOIN BOW ON df2.GrievanceDesc_clean LIKE '%' || BOW.list || '%'"""
df3 = pysqldf(q)

df3.loc[:, ['list']] = df3[['list']].notnull() * 1

df3.drop_duplicates(subset=None, keep='first', inplace=True)
df3 = df3.rename(columns={'list': 'UFDP'})

print("Look at the distribution")
print(df3['UFDP'].value_counts())

df3.reset_index()
df3=df3.reset_index(drop=True)
frames = [FeatureData, df3['UFDP']]
df2 = pd.concat(frames,axis=1)
FeatureData = pd.concat(frames,axis=1)

Look at the distribution
1    13331
0    10834
Name: UFDP, dtype: int64


In [67]:
##Final prepared data at grievance level -test
Add_data=  testdf1[['BankID','State','DateOfGrievance','Grievance_Category','DateOfResolution']]
frames = [FeatureData, Add_data]
FinalGrievanceDt = pd.concat(frames,axis=1)

In [69]:
FinalGrievanceDt.shape

(24165, 24)

In [70]:
#Derive days to resolve

FinalGrievanceDt['Daystoresolve']=((pd.to_datetime(FinalGrievanceDt['DateOfResolution']))-(pd.to_datetime(FinalGrievanceDt['DateOfGrievance']))).dt.days

In [73]:
#### Creating 5 additional features flags for top 5 (highest grievance issues) states

FinalGrievanceDt['State9'] = np.where(FinalGrievanceDt['State']=='State9', 1, 0)
FinalGrievanceDt['State14'] = np.where(FinalGrievanceDt['State']=='State14', 1, 0)
FinalGrievanceDt['State43'] = np.where(FinalGrievanceDt['State']=='State43', 1, 0)
FinalGrievanceDt['State53'] = np.where(FinalGrievanceDt['State']=='State53', 1, 0)
FinalGrievanceDt['State47'] = np.where(FinalGrievanceDt['State']=='State47', 1, 0)

In [74]:
#### Creating 2 variables using line of business for mortgage and credit cards

FinalGrievanceDt['Mortgage'] = np.where(FinalGrievanceDt['LineOfBusiness']=='Mortgage', 1, 0)
FinalGrievanceDt['CreditCard'] = np.where(FinalGrievanceDt['LineOfBusiness']=='Credit card', 1, 0)

In [76]:
#### Creating 4 additional variables for days to resolve 
FinalGrievanceDt['DaystoResolveflg']='NA'
FinalGrievanceDt['DaystoResolveflg'][(FinalGrievanceDt['Daystoresolve'] <= 30)] = '1Monthorless'
FinalGrievanceDt['DaystoResolveflg'][(FinalGrievanceDt['Daystoresolve'] > 30) & (FinalGrievanceDt['Daystoresolve'] <= 60)] = '1to2Months'
FinalGrievanceDt['DaystoResolveflg'][(FinalGrievanceDt['Daystoresolve'] > 60) & (FinalGrievanceDt['Daystoresolve'] <= 90)] = '2to3Months'
FinalGrievanceDt['DaystoResolveflg'][(FinalGrievanceDt['Daystoresolve'] > 90)] = '3Monthsormore'

FinalGrievanceDt['Lessthan1Month'] = np.where(FinalGrievanceDt['DaystoResolveflg']=='1Monthorless', 1, 0)
FinalGrievanceDt['1to2Months'] = np.where(FinalGrievanceDt['DaystoResolveflg']=='1to2Months', 1, 0)
FinalGrievanceDt['2to3Months'] = np.where(FinalGrievanceDt['DaystoResolveflg']=='2to3Months', 1, 0)
FinalGrievanceDt['3Monthsormore'] = np.where(FinalGrievanceDt['DaystoResolveflg']=='3Monthsormore', 1, 0)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda2/lib/python2.7/site-packages

In [78]:
#### Create 1 additional variable for closed with explanation grievances 

FinalGrievanceDt['ClosedwithExplain'] = np.where(FinalGrievanceDt['ResolutionComments']=='Closed with explanation', 1, 0)

In [79]:
#### Creating 4 additional variables for days to resolve 

FinalGrievanceDt['ClosedwithExplain_Dispute'] = np.where((FinalGrievanceDt['ResolutionComments']=='Closed with explanation') & (FinalGrievanceDt['Disputed']=='Yes'), 1, 0)

In [80]:
#### Create 1 additional variable for closed with explanation grievances 
FinalGrievanceDt['ClosedwithExplain_Dispute_Commu'] = np.where((FinalGrievanceDt['ResolutionComments']=='Closed with explanation') & (FinalGrievanceDt['Disputed']=='Yes') & (FinalGrievanceDt['Communication']==1), 1, 0)

In [98]:
FinalGrievanceDt.shape

(24165, 40)

In [100]:
#Save the data to pickle
#FinalGrievanceDt.to_pickle('TestFinalGrievanceDt_Saved.pkl') 
FinalGrievanceDt = pd.read_pickle('TestFinalGrievanceDt_Saved.pkl')

In [ ]:
##Data Role up at bank level

In [128]:
State9 = FinalGrievanceDt[['BankID','State9']].groupby(['BankID'])['State9'].sum().reset_index(name='State9_sum')
State14 = FinalGrievanceDt[['BankID','State14']].groupby(['BankID'])['State14'].sum().reset_index(name='State14_sum')
State43 = FinalGrievanceDt[['BankID','State43']].groupby(['BankID'])['State43'].sum().reset_index(name='State43_sum')
State53 = FinalGrievanceDt[['BankID','State53']].groupby(['BankID'])['State53'].sum().reset_index(name='State53_sum')
State47 = FinalGrievanceDt[['BankID','State47']].groupby(['BankID'])['State47'].sum().reset_index(name='State47_sum')
BusinessPractices = FinalGrievanceDt[['BankID','BusinessPractices']].groupby(['BankID'])['BusinessPractices'].sum().reset_index(name='BusinessPract_Sum')
Communication = FinalGrievanceDt[['BankID','Communication']].groupby(['BankID'])['Communication'].sum().reset_index(name='Communicn_Sum')
CIBILScore = FinalGrievanceDt[['BankID','CIBILScore']].groupby(['BankID'])['CIBILScore'].sum().reset_index(name='CIBILScore_Sum')
LateFeeInterestCharges = FinalGrievanceDt[['BankID','LateFeeInterestCharges']].groupby(['BankID'])['LateFeeInterestCharges'].sum().reset_index(name='LateFeeIntCharg_Sum')
CloseCancel = FinalGrievanceDt[['BankID','CloseCancel']].groupby(['BankID'])['CloseCancel'].sum().reset_index(name='CloseCancel_Sum')
purchase = FinalGrievanceDt[['BankID','purchase']].groupby(['BankID'])['purchase'].sum().reset_index(name='purchaser_Sum')
DelayRecur = FinalGrievanceDt[['BankID','DelayRecur']].groupby(['BankID'])['DelayRecur'].sum().reset_index(name='DelayRecur_Sum')
legal = FinalGrievanceDt[['BankID','legal']].groupby(['BankID'])['legal'].sum().reset_index(name='legal_Sum')
Mkting_Offer_Reward = FinalGrievanceDt[['BankID','Mkting_Offer_Reward']].groupby(['BankID'])['Mkting_Offer_Reward'].sum().reset_index(name='MktOfferRwd_Sum')
PaymentIssue = FinalGrievanceDt[['BankID','PaymentIssue']].groupby(['BankID'])['PaymentIssue'].sum().reset_index(name='PaymentIssue_Sum')
MortgageBroadly = FinalGrievanceDt[['BankID','MortgageBroadly']].groupby(['BankID'])['MortgageBroadly'].sum().reset_index(name='MortgageBroad_Sum')
TeftFraudThreat = FinalGrievanceDt[['BankID','TeftFraudThreat']].groupby(['BankID'])['TeftFraudThreat'].sum().reset_index(name='TeftFraudThreat_Sum')
Mortgage = FinalGrievanceDt[['BankID','Mortgage']].groupby(['BankID'])['Mortgage'].sum().reset_index(name='Mortgage_Sum')
CreditCard = FinalGrievanceDt[['BankID','CreditCard']].groupby(['BankID'])['CreditCard'].sum().reset_index(name='CreditCard_Sum')
Lessthan1Month = FinalGrievanceDt[['BankID','Lessthan1Month']].groupby(['BankID'])['Lessthan1Month'].sum().reset_index(name='Lessthan1Month_Sum')
OnetotwoMonths = FinalGrievanceDt[['BankID','1to2Months']].groupby(['BankID'])['1to2Months'].sum().reset_index(name='1to2Months_Sum')
twotothreeMonths = FinalGrievanceDt[['BankID','2to3Months']].groupby(['BankID'])['2to3Months'].sum().reset_index(name='2to3Months_Sum')
More3months = FinalGrievanceDt[['BankID','3Monthsormore']].groupby(['BankID'])['3Monthsormore'].sum().reset_index(name='3Monthsormore_Sum')
ClosExpl = FinalGrievanceDt[['BankID','ClosedwithExplain']].groupby(['BankID'])['ClosedwithExplain'].sum().reset_index(name='ClosExpl_Sum')
ClosExplDisp = FinalGrievanceDt[['BankID','ClosedwithExplain_Dispute']].groupby(['BankID'])['ClosedwithExplain_Dispute'].sum().reset_index(name='ClosExplDisp_Sum')
ClosExplDispCom = FinalGrievanceDt[['BankID','ClosedwithExplain_Dispute_Commu']].groupby(['BankID'])['ClosedwithExplain_Dispute_Commu'].sum().reset_index(name='ClosExplDispCom_Sum')
CntGriev = FinalGrievanceDt[['BankID','GrievanceID']].groupby(['BankID'])['GrievanceID'].count().reset_index(name='Cnt_Griev')
AvgdaystoRes = FinalGrievanceDt[['BankID','Daystoresolve']].groupby(['BankID'])['Daystoresolve'].mean().reset_index(name='AvgdaystoResolve')
MeddaystoRes = FinalGrievanceDt[['BankID','Daystoresolve']].groupby(['BankID'])['Daystoresolve'].median().reset_index(name='MediandaystoResolve')
Payments = FinalGrievanceDt[['BankID','Payments']].groupby(['BankID'])['Payments'].sum().reset_index(name='Payments_Sum')
UFDP = FinalGrievanceDt[['BankID','UFDP']].groupby(['BankID'])['UFDP'].sum().reset_index(name='UFDP_Sum')

In [129]:
State9.set_index("BankID",drop=True,inplace=True)
State14.set_index("BankID",drop=True,inplace=True)
State43.set_index("BankID",drop=True,inplace=True)
State53.set_index("BankID",drop=True,inplace=True)
State47.set_index("BankID",drop=True,inplace=True)
BusinessPractices.set_index("BankID",drop=True,inplace=True)
Communication.set_index("BankID",drop=True,inplace=True)
CIBILScore.set_index("BankID",drop=True,inplace=True)
LateFeeInterestCharges.set_index("BankID",drop=True,inplace=True)
CloseCancel.set_index("BankID",drop=True,inplace=True)
purchase.set_index("BankID",drop=True,inplace=True)
DelayRecur.set_index("BankID",drop=True,inplace=True)
legal.set_index("BankID",drop=True,inplace=True)
Mkting_Offer_Reward.set_index("BankID",drop=True,inplace=True)
PaymentIssue.set_index("BankID",drop=True,inplace=True)
MortgageBroadly.set_index("BankID",drop=True,inplace=True)
TeftFraudThreat.set_index("BankID",drop=True,inplace=True)
Mortgage.set_index("BankID",drop=True,inplace=True)
CreditCard.set_index("BankID",drop=True,inplace=True)
Lessthan1Month.set_index("BankID",drop=True,inplace=True)
OnetotwoMonths.set_index("BankID",drop=True,inplace=True)
twotothreeMonths.set_index("BankID",drop=True,inplace=True)
More3months.set_index("BankID",drop=True,inplace=True)
ClosExpl.set_index("BankID",drop=True,inplace=True)
ClosExplDisp.set_index("BankID",drop=True,inplace=True)
ClosExplDispCom.set_index("BankID",drop=True,inplace=True)
CntGriev.set_index("BankID",drop=True,inplace=True)
AvgdaystoRes.set_index("BankID",drop=True,inplace=True)
MeddaystoRes.set_index("BankID",drop=True,inplace=True)
Payments.set_index("BankID",drop=True,inplace=True)
UFDP.set_index("BankID",drop=True,inplace=True)

In [130]:
## 1) Import Bank level Target variable
BankID=pd.read_csv('BankTestID.csv')

In [131]:
frames = [State9,State14,State43,State53,State47,BusinessPractices,Communication,CIBILScore,LateFeeInterestCharges,CloseCancel,purchase,DelayRecur,legal,Mkting_Offer_Reward,PaymentIssue,MortgageBroadly,TeftFraudThreat,Mortgage,CreditCard,Lessthan1Month,OnetotwoMonths,twotothreeMonths,More3months,ClosExpl,ClosExplDisp,ClosExplDispCom,CntGriev,AvgdaystoRes,MeddaystoRes,Payments,UFDP]
Bankleveldata= pd.concat(frames,axis=1)

FinalBankleveldata = pd.merge(Bankleveldata, BankID, how='inner', on=['BankID'])

In [132]:
FinalBankleveldata['State9_pct'] = ((FinalBankleveldata['State9_sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['State14_pct'] = ((FinalBankleveldata['State14_sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['State43_pct'] = ((FinalBankleveldata['State43_sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['State53_pct'] = ((FinalBankleveldata['State53_sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['State47_pct'] = ((FinalBankleveldata['State47_sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)            
FinalBankleveldata['BusinessPract_pct'] = ((FinalBankleveldata['BusinessPract_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['Communicn_pct'] = ((FinalBankleveldata['Communicn_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['LateFeeIntCharg_pct'] = ((FinalBankleveldata['LateFeeIntCharg_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                  
FinalBankleveldata['CIBILScore_pct'] = ((FinalBankleveldata['CIBILScore_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['CloseCancel_pct'] = ((FinalBankleveldata['CloseCancel_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['purchaser_pct'] = ((FinalBankleveldata['purchaser_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['DelayRecur_pct'] = ((FinalBankleveldata['DelayRecur_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['legal_pct'] = ((FinalBankleveldata['legal_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['MktOfferRwd_pct'] = ((FinalBankleveldata['MktOfferRwd_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                      
FinalBankleveldata['PaymentIssue_pct'] = ((FinalBankleveldata['PaymentIssue_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)
FinalBankleveldata['MortgageBroad_pct'] = ((FinalBankleveldata['MortgageBroad_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)        
FinalBankleveldata['TeftFraudThreat_pct'] = ((FinalBankleveldata['TeftFraudThreat_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)    
FinalBankleveldata['Mortgage_pct'] = ((FinalBankleveldata['Mortgage_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                    
FinalBankleveldata['CreditCard_pct'] = ((FinalBankleveldata['CreditCard_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                    
FinalBankleveldata['Lessthan1Month_pct'] = ((FinalBankleveldata['Lessthan1Month_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                    
FinalBankleveldata['onetotwoMonths_pct'] = ((FinalBankleveldata['1to2Months_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                    
FinalBankleveldata['twotothreeMonths_pct'] = ((FinalBankleveldata['2to3Months_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                    
FinalBankleveldata['threeormoreMonths_pct'] = ((FinalBankleveldata['3Monthsormore_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                    
FinalBankleveldata['ClosExpl_pct'] = ((FinalBankleveldata['ClosExpl_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                    
FinalBankleveldata['ClosExplDisp_pct'] = ((FinalBankleveldata['ClosExplDisp_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                    
FinalBankleveldata['ClosExplDispCom_pct'] = ((FinalBankleveldata['ClosExplDispCom_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)    
FinalBankleveldata['Payments_pct'] = ((FinalBankleveldata['Payments_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)          
FinalBankleveldata['UFDP_pct'] = ((FinalBankleveldata['UFDP_Sum']/FinalBankleveldata['Cnt_Griev'])*100).round(2)                                                       
                  

In [133]:
##Final Test data
FnllBanklvldata_Tgt= FinalBankleveldata.loc[:,['BankID','AvgdaystoResolve','MediandaystoResolve','State9_pct','State14_pct','State43_pct','State53_pct','State47_pct','BusinessPract_pct','Communicn_pct','LateFeeIntCharg_pct','CIBILScore_pct','CloseCancel_pct','purchaser_pct','DelayRecur_pct','legal_pct','MktOfferRwd_pct','PaymentIssue_pct','MortgageBroad_pct','TeftFraudThreat_pct','Mortgage_pct','CreditCard_pct','Lessthan1Month_pct','onetotwoMonths_pct','twotothreeMonths_pct','threeormoreMonths_pct','ClosExpl_pct','ClosExplDisp_pct','ClosExplDispCom_pct','Payments_pct','UFDP_pct']]

In [135]:
FnllBanklvldata_Tgt.shape

(22, 31)

In [138]:
#Save the final bank level aggregated data to pickle

#FnllBanklvldata_Tgt.to_pickle('TestFnllBanklvldata_Tgt_Saved.pkl') 
FnllBanklvldata_Tgt = pd.read_pickle('TestFnllBanklvldata_Tgt_Saved.pkl')

In [136]:
FnllBanklvldata_Tgt.describe()

,AvgdaystoResolve,MediandaystoResolve,State9_pct,State14_pct,State43_pct,State53_pct,State47_pct,BusinessPract_pct,Communicn_pct,LateFeeIntCharg_pct,...,CreditCard_pct,Lessthan1Month_pct,onetotwoMonths_pct,twotothreeMonths_pct,threeormoreMonths_pct,ClosExpl_pct,ClosExplDisp_pct,ClosExplDispCom_pct,Payments_pct,UFDP_pct
count,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,...,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,17.763099,13.613636,14.854091,9.196364,6.235455,7.064091,4.739091,20.985000,62.080909,26.646364,...,31.109545,87.409091,12.062727,0.390909,0.135000,85.093636,20.095000,13.589091,25.695455,58.682273
std,3.475868,5.126801,7.645566,3.661246,3.417582,4.340198,3.973244,7.382058,8.370914,8.425706,...,43.370517,13.650635,13.191300,0.659660,0.379257,16.105825,6.622509,5.149537,9.576080,6.681521
min,14.266010,11.000000,0.000000,2.200000,0.000000,1.100000,0.920000,8.000000,43.330000,3.330000,...,0.000000,46.840000,0.920000,0.000000,0.000000,55.270000,10.990000,6.100000,11.380000,46.940000
25%,15.659107,11.000000,11.572500,7.602500,4.332500,3.497500,2.825000,17.135000,57.665000,25.487500,...,0.000000,87.560000,4.160000,0.000000,0.000000,72.207500,15.085000,9.372500,18.300000,54.047500
50%,17.113172,11.000000,13.710000,8.510000,5.665000,6.880000,3.945000,20.510000,64.850000,29.235000,...,0.000000,91.990000,7.845000,0.095000,0.000000,94.040000,18.675000,12.955000,27.550000,59.545000
75%,18.333681,13.750000,17.022500,10.120000,8.297500,9.730000,4.967500,25.452500,66.657500,31.462500,...,79.345000,95.555000,12.440000,0.557500,0.030000,97.190000,22.775000,16.215000,30.797500,62.877500
max,29.063291,31.000000,31.650000,17.950000,13.190000,16.340000,19.780000,36.020000,75.950000,37.660000,...,100.000000,99.080000,50.630000,2.530000,1.670000,100.000000,33.710000,24.050000,48.320000,72.370000


In [137]:
print(FnllBanklvldata_Tgt.isnull().sum())

BankID                   0
AvgdaystoResolve         0
MediandaystoResolve      0
State9_pct               0
State14_pct              0
State43_pct              0
State53_pct              0
State47_pct              0
BusinessPract_pct        0
Communicn_pct            0
LateFeeIntCharg_pct      0
CIBILScore_pct           0
CloseCancel_pct          0
purchaser_pct            0
DelayRecur_pct           0
legal_pct                0
MktOfferRwd_pct          0
PaymentIssue_pct         0
MortgageBroad_pct        0
TeftFraudThreat_pct      0
Mortgage_pct             0
CreditCard_pct           0
Lessthan1Month_pct       0
onetotwoMonths_pct       0
twotothreeMonths_pct     0
threeormoreMonths_pct    0
ClosExpl_pct             0
ClosExplDisp_pct         0
ClosExplDispCom_pct      0
Payments_pct             0
UFDP_pct                 0
dtype: int64


In [ ]:
dropc = ['ClosExpl_pct','ClosExplDispCom_pct','MortgageBroad_pct','MediandaystoResolve','Lessthan1Month_pct','onetotwoMonths_pct','BankID','State53_pct','twotothreeMonths_pct','State14_pct','Communicn_pct','purchaser_pct','State47_pct','threeormoreMonths_pct']
Testdata=FnllBanklvldata_Tgt.drop(dropc, axis=1)